In this tutorial, will go through the whole tensorflow application working procedures step by step. And we'll use a MNIST dataset as the example. Most of this note is built on analysis of 'mnist.py' and 'fully_connected_feed.py' under /tensorflow/examples/tutorial/mnist path.

h1. Prepare data
MNIST is a classic problem in ML. It is to look at greyscale 28 x 28 pixel images of handwritten digits and determine which digit the image represents, for all the digits from zero to nine.

In [ ]:
'''
Builds the MNIST network.
Implements the inference/loss/training pattern for model building.
1. inference() - Builds the model as far as is required for running the network forward to make predictions;
2. loss() - Adds to the inference model the layers required to generate loss;
3. training() - Adds to the loss model the Ops required to generate and apply gradients.
'''

def inference(images, hidden1_units, hidden2_units):
    '''Builds the MNIST model up to where it may be used for inference.'''
    #Hidden 1
    with tf.name_scope('hidden1'):
        weigths = tf.Variable(
            tf.truncated_normal([IMAGE_PIXELS, hidden1_units], stddev = 1.0 / math.sqrt(float(IMAGE_PIXELS))), name = 'weights')
        biases = tf.Variable(tf.zeros([hidden1_units]), name = 'biases')
        hidden1 = tf.nn.relu(tf.matmul(images, weights) + biases)
    
    #Hidden 2
    with tf.name_scope('hidden2'):
        weigths = tf.Variable(
            tf.truncated_normal([hidden1_units, hidden2_units], stddev = 1.0 / math.sqrt(float(hidden1_units))), name = 'weights')
        biases = tf.Variable(tf.zeros([hidden2_units]), name = 'biases')
        hidden2 = tf.nn.relu(tf.matmul(hidden1, weights) + biases)
    
    #Linear
    with tf.name_scope('softmax_linear'):
        weigths = tf.Variable(
            tf.truncated_normal([hidden2_units, NUM_CLASSES], stddev = 1.0 / math.sqrt(float(hidden2_units))), name = 'weights')
        biases = tf.Variable(tf.zeros([NUM_CLASSES]), name = 'biases')
        logits = tf.nn.relu(tf.matmul(hidden2, weights) + biases)
        
    return logits

def loss(logits, labels):
    '''Calculates the loss from the logits and the labels'''
    labels = tf.to_int64(labels)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
        labels = labels, logits = logits, name = 'xentropy')
    return tf.reduce_mean(cross_entropy, name = 'xentropy_mean')

def training(loss, learning_rate):
    '''Sets up the training ops.
    Creates a summarizer to track the loss over time in Tensorboard.
    Creates an optimizer and applies the gradients to all trainable variables.
    The Op returned by this function is what must be passed to the 'see.run()' call to cause the model to train.
    '''
    #Add a scalar summary for the snapshot loss.
    tf.summary.scalar('loss', loss)
    #Create the gradient descent optimizer with the given learning rate
    optimizer = tf.train.GradientDescentOptimization(learning_rate)
    #Create a variable to track the global step.
    global_step = tf.Variable(0, name = 'global_step', trainable = False)
    #Use the optimizer to apply the gradients that minimize the loss, also increment the global_step counter
    #as a single training step.
    train_op = optimizer.minimize(loss, global_step = global_step)
    return train_op

def evaluation(logits, labels):
    '''Evaluate the quality of the logits at predicting the label'''
    correct = tf.nn.in_top_k(logits, labels, 1)
    #Return the number of true entries.
    return tf.reduce_sum(tf.cast(correct, tf.int32))

######Below are functions defined with 'fully_connected_feed.py'
# Trains and evaluates the MNIST network using a feed dictionary.
def do_eval(sess,
           eval_correct,
           images_placeholder,
           labels_placeholder,
           data_set):
    '''Runs one evaluation against the full epoch of data.'''
    true_count = 0
    steps_per_epoch = dataset.num_examples // FLAGS.batch_size
    num_examples = steps_per_epoch * FLAGS.batch_size
    for step in xrange(steps_per_epoch):
        feed_dict = fill_feed_dict(data_set,
                                  images_placeholder,
                                  labels_placeholder)
        true_count += sess.run(eval_correct, feed_dict = feed_dict)
    precision = float(ture_count) / num_examples
    
def run_training():
    '''Train MNIST for a number of steps.'''
    #Gets the sets of images and labels
    data_sets = input_data.read_data_sets(FLAGS.input_data_dir, FLAGS.fake_data)
    
    #The model will be built into the default graph.
    with tf.Graph().as_default():
        #Generate placeholders
        images_placeholder, labels_placeholder = placeholder_inputs(FLAGS.batch_size)
        
        #Build a graph that computes predictions from the inference model.
        logits = mnist.inference(images_placeholder,
                                FLAGS.hidden1,
                                FLAGS.hidden2)
        
        #Add to the graph the Ops for loss calculation.
        loss = mnist.loss(logits, labels_placeholder)
        
        #Add to the graph the ops that calculate and apply gradients
        train_op = mnist.training(loss, FLAGS.learning_rate)
        
        #Add the op to compare the logits to the labels during evaluation
        eval_correct = mnist.evaluation(logits, labels_placeholder)
        
        #Build the summary Tensor based on the TF collection of Summaries.
        summary = tf.summary.merge_all()
        
        #Add the variable initializer Op.
        init = tf.global_variables_initializer()
        
        #Create a saver for writing training checkpoints.
        save = tf.train.Saver()
        
        #Create a session for running Ops on the Graph.
        sess = tf.Session()
        
        #Instantiate a summaryWriter to output summaries and the graph.
        summary_writer = tf.summary.FileWriter(FLAGS.log_dir, sess.graph)
        
        #Now everything is built on the graph.
        
        #Run the Op to initialize the variables.
        sess.run(init)
        
        #Start the training loops.
        for step in xrange(FLAGS.max_steps):
            start_time = time.time()
            
            #Fill a feed dcit
            feed_dict = fill_feed_dict(data_sets.train,
                                      images_placeholder,
                                      labels_placeholder)
            
            #Run one step of the model
            _, loss_value = sess.run([train_op, loss], feed_dict = feed_dict)
            duration = time.time - start_time
            
            #Write the summaries and print an overview
            if step % 100 == 0:
                #print status to stdout.
                print('Step %d: loss = %.2f (%.3f sec)' % (step, loss_value, duration))
                #Update the event files
                summary_str = sess.run(summary, feed_dict = feed_dict)
                summary_writer.add_summary(summary_str, step)
                summary_writer.flush()
                
            #Save a checkpoint and evaluate the model periodically
            if(step + 1) % 1000 == 0 or (step + 1) == FLAGS.max_steps:
                #Emit a checkpoint file that maybe used to later restore a model for further training or evaluation.
                checkpoint_file = os.path.join(FLAGS.log_dir, 'model.ckpt')
                #save is called to write a checkpoint file to the training directory with the current values of
                #all the trainable variables.
                #In the future, training might be resumed by using the tf.train.Saver.restore method to reload the model
                #parameters.
                saver.save(sess, checkpoint_file, global_step = step)
                #Evaluate against the training set.
                print('Training data eval:')
                do_eval(sess, 
                       eval_correct,
                       images_placeholder,
                       labels_placeholder,
                       data_sets.train)
                #Evaluate against the validation set.
                print('Validation data eval:')
                do_eval(sess,
                       eval_correct,
                       images_placeholder,
                       labels_placeholder,
                       data_sets.validation)
                #Evaluate against the test set.
                print('Test data eval:')
                do_eval(sess,
                       eval_correct,
                       images_placeholder,
                       labels_placeholder,
                       data_sets.test)